In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
def plot_spectrogram(spec, freqs, max_freq=5000, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
        fig.set_size_inches(10, 5)
        
    ax.pcolorfast(range(spec.shape[1]), freqs[freqs<max_freq], np.log10(spec[freqs<max_freq]))
    ax.set_ylabel('frequency [Hz]')
    return ax

# First study

In [ ]:
from evaluate_data import read_df_from_wav, get_spectrogram

freq = 1750
position_dict = {
    3: 'upper',
    #4: 'upper',
    5: 'current',
    #6: 'current',
    7: 'lower',
    #8: 'lower',
    14: 'lowest',
    #15: 'lowest',
}

In [ ]:
base_filename = f'../experiments/2021_01_07_snr_study/export/motors_nosnr_noprops_mono{freq}'

df_specs = pd.DataFrame(columns=['number', 'spec', 'freqs'])
for number, position_name in position_dict.items():
    appendix = '' if number==0 else f'_{number}'
    try:
        filename = f'{base_filename}{appendix}.wav'
        df_wav = read_df_from_wav(filename)
    except FileNotFoundError:
        print('skipping', filename)
        continue
    spec = get_spectrogram(df_wav)
    freqs = df_wav.iloc[0].frequencies
    
    ax = plot_spectrogram(spec, freqs, max_freq=5000)
    ax.set_title(position_name)
    
    df_specs.loc[len(df_specs), :] = {
        'number': number,
        'spec':spec,
        'freqs':freqs
    }

In [ ]:
time_window_dict =  {
    'buzzer only': (range(10, 20), '-'),
    'buzzer and props': (range(80, 90), ':')
}
max_freq = 16000

fig, axs = plt.subplots(1, len(df_specs), sharey=True, squeeze=False)
[ax.grid(which='both') for ax in axs.flatten()]
fig.set_size_inches(20, 5)
for title, (time_window, ls) in time_window_dict.items():
    
    fig, ax  = plt.subplots()
    fig.set_size_inches(20, 5)
    
    for i, row in df_specs.iterrows():
        spec = row.spec
        freqs = row.freqs


        spec_window = spec[:, time_window] 

        ax.semilogy(freqs, np.mean(spec_window, axis=1), label=position_dict[row.number], color=f'C{i}')
        axs[0, i].semilogy(freqs, np.mean(spec_window, axis=1), color=f'C{i}', ls=ls, label=title)
        axs[0, i].set_xlim(0, max_freq)
        axs[0, i].set_title(position_dict[row.number])
        
    axs[0, i].legend(loc='upper right')
    axs[0, i].semilogy()
    
    ax.legend(loc='upper right')
    ax.set_title(title)
    ax.grid(which='both')
    ax.set_xlim(0, max_freq)

In [ ]:
fig = plt.figure()
fig.set_size_inches(15, 15)
plt.loglog(freqs, np.mean(spec_window, axis=1), color=f'C{i}')
plt.grid(which='both')

# Second study

In [ ]:
exp_dict = {
    '45000': 'motors_nosnr_noprops_None.wav',
    '55000': 'motors_nosnr_noprops_None_55000.wav',
    'mono': 'nomotors_nosnr_noprops_mono3125.wav',
    'mono 55000': 'motors_nosnr_noprops_mono3125_55000.wav',
    'mono 45000': 'motors_nosnr_noprops_mono3125.wav',
    'sweep': 'nomotors_nosnr_noprops_sweep.wav',
    'sweep 45000': 'motors_nosnr_noprops_sweep.wav',
}

freq = 3125
#base_filename = f'../experiments/2021_01_14_snr_study/export/'
#base_filename = f'../experiments/2021_01_15_snr_study/export/'
base_filename = f'../experiments/2021_01_15_snr_study_2/export/'

df_specs = pd.DataFrame(columns=['title', 'spec', 'freqs'])
for title, fname in exp_dict.items():
    try:
        filename = base_filename + fname
        df_wav = read_df_from_wav(filename)
    except FileNotFoundError:
        print('skipping', filename)
        continue
    spec = get_spectrogram(df_wav)
    freqs = df_wav.iloc[0].frequencies
    
    ax = plot_spectrogram(spec, freqs, max_freq=5000)
    ax.set_title(title)
    
    df_specs.loc[len(df_specs), :] = {
        'title': title,
        'spec':spec,
        'freqs':freqs
    }

In [ ]:
title_list = ['mono', '55000', 'mono 55000']
title_list = ['mono', '45000', 'mono 45000']
ls_list = [':', ':', ':']
start_time = 67
max_freq = 5000
min_freq = 100

fig, ax = plt.subplots()
for title, ls in zip(title_list, ls_list):
    row = df_specs.loc[df_specs.title==title].iloc[0]
    
    spec = row.spec[:, start_time:]
    freqs = row.freqs
    #ax_spec = plot_spectrogram(spec, row.freqs)
    #ax_spec.set_title(title)
    #ax.loglog(row.freqs, np.mean(spec, axis=1), label=title)
    ax.semilogy(freqs, np.mean(spec, axis=1), label=title, ls=ls)
ax.set_title('average amplitude')
ax.legend(loc='upper right')
ax.set_xlabel('frequency [Hz]')
ax.set_xlim(min_freq, max_freq)

fig, ax = plt.subplots()
spec_monoprops = df_specs.loc[df_specs.title=='mono 45000'].iloc[0].spec[:, start_time:]
spec_props = df_specs.loc[df_specs.title=='45000'].iloc[0].spec[:, start_time:]
ax.semilogy(freqs, np.mean(spec_monoprops, axis=1)/np.mean(spec_props, axis=1), label='ratio')
ax.legend(loc='upper right')
ax.set_xlabel('frequency [Hz]')
ax.set_title('ratio of mono 45000 vs. 45000')
ax.set_xlim(min_freq, max_freq)

# propeller noise study

In [ ]:
for thrust in [45000, 55000]:
    row = df_specs.loc[df_specs.title==str(thrust)].iloc[0]
    spec = row.spec[:, start_time:]
    
    mean = np.mean(spec, axis=1)
    std = np.std(spec, axis=1)
    
    plt.figure()
    plt.semilogy(row.freqs, mean)
    plt.semilogy(row.freqs, mean+std)
    plt.semilogy(row.freqs, std/mean)
    #plt.errorbar(x=row.freqs, y=mean, yerr=1.0) #yerr=std)
    plt.xlim(100, 5000)
    plt.ylim(1e-3, 1e4)
    plt.title(f'motor thrust: {thrust}')

# sweep study

In [ ]:
row_sweep = df_specs.loc[df_specs.title=="sweep"].iloc[0]
max_bins = np.argmax(row_sweep.spec, axis=0)
max_freqs = row_sweep.freqs[max_bins]

#end_freq = max_freqs[np.argmin(np.abs(max_freqs - 4875))]
#end_freq_idx = np.argmin(np.abs(row.freqs - end_freq))
#end_idx = np.where(max_freqs == end_freq)[0][-1]

start_freq = max_freqs[np.argmin(np.abs(max_freqs - 3000))]
start_idx = np.where(max_freqs == start_freq)[0][0]
print('start idx', start_idx)

end_idx = np.where(max_freqs > 2000)[0][-1]

plt.figure()
#plt.plot(max_freqs[start_idx:end_idx])
plt.plot(max_freqs)
plt.axvline(start_idx, color='k')
plt.axvline(end_idx, color='k')

max_bins = max_bins[start_idx:end_idx]
end_idx -= start_idx

plt.figure()
plt.plot(max_bins)

In [ ]:
start = 250 
unique_bins = np.unique(max_bins[start:])

fig, ax_all = plt.subplots()

fig, axs = plt.subplots(len(unique_bins), sharey=True, sharex=True)
fig.set_size_inches(10, len(unique_bins)*3)
for j, title in enumerate(["sweep", "sweep 45000"]):
    row = df_specs.loc[df_specs.title==title].iloc[0]

    max_freqs = row.freqs[np.argmax(row.spec, axis=0)]
    start_freq = max_freqs[np.argmin(np.abs(max_freqs - 3000))]
    start_idx = np.where(max_freqs == start_freq)[0][0]

    spec = row.spec[:, start_idx:]
    freqs = row.freqs
    times = range(spec.shape[1])

    ax = plot_spectrogram(spec, freqs, max_freq=5000)
    ax.set_title(title)

    max_freqs = row.freqs[max_bins]
    ax.plot(range(len(max_freqs)), max_freqs, color='white', ls=':')
    ax.set_ylim(1000, 5000)
    
    ax.axvline(start, color='white', ls='-')
    
    max_amps = {}
    for i, bin_ in enumerate(unique_bins):

        freq = np.round(freqs[bin_])
        # start from 1 to remove "edge" values
        cut_indices = np.where(max_bins[start:]==bin_)[0][1:]
        if not len(cut_indices):
            continue

        cut_spec = spec[:, start+cut_indices]
        avg_spec = np.median(cut_spec, axis=1)
        std_spec = np.std(cut_spec[bin_]) / avg_spec[bin_] * 100
        
        if len(cut_indices) >= 10:
            max_amps[freq] = (avg_spec[bin_], std_spec)
        axs[i].semilogy(freqs, avg_spec, ls=ls, label=f'std ratio: {std_spec:.0f}')
        axs[i].legend(loc='upper right')
        axs[i].set_title(f'{freq} Hz: average over {len(cut_indices)}')
        axs[i].grid(which='major')
    axs[i].set_ylim(1e-3, 1e3)
    axs[i].set_xlim(1000, 5000)
    
    means = [v[0] for v in max_amps.values()]
    stds = [v[1] for v in max_amps.values()]
    ax_all.errorbar(list(max_amps.keys()), means, stds, marker='o', label=title+' mean', ls='-', color=f'C{j}')
    #ax_all.semilogy(list(max_amps.keys()), stds, marker='o', label=title+' std', ls=':', color=f'C{j}')
    ax_all.set_yscale('log')
ax_all.legend()